In [2]:
!pip install spotipy


In [4]:
pip install faiss-cpu


   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/13.8 MB 4.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/13.8 MB 4.7 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/13.8 MB 4.5 MB/s eta 0:00:03
   -------- ------------------------------- 2.9/13.8 MB 3.7 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.8 MB 3.1 MB/s eta 0:00:04
   --------- ------------------------------ 3.1/13.8 MB 3.1 MB/s eta 0:00:04
   ---------- ----------------------------- 3.7/13.8 MB 2.6 MB/s eta 0:00:04
   ------------ --------------------------- 4.5/13.8 MB 2.7 MB/s eta 0:00:04
   -------------- ------------------------- 5.0/13.8 MB 2.7 MB/s eta 0:00:04
   --------------- ------------------------ 5.5/13.8 MB 2.6 MB/s eta 0:00:04
   ---------------- ----------------------- 5.8/13.8 MB 2.6 MB/s eta 0:00:04
   ------------------ --------------------- 6.3/13.8 MB 2.6 MB/s eta 0:00:03
   ---

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import faiss
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import display, Image

# Spotify API Credentials
CLIENT_ID = "4da10aae52c04c8ab5c4a5300212c9d5"
CLIENT_SECRET = "cfc4944866a943c9984b87cce8167b91"

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Load dataset
DATASET_PATH = r"C:\Users\Asmi\Downloads\archive\dataset.csv"
music = pd.read_csv(DATASET_PATH)

# Required columns
required_columns = ['track_name', 'artists', 'danceability', 'energy', 'valence', 
                    'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']

# Preprocessing for Similarity Calculation
def build_faiss_index(df):
    features = required_columns[2:]  # Exclude 'track_name' and 'artists'
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    # Build a FAISS index
    d = scaled_features.shape[1]  # Dimensionality of vectors
    index = faiss.IndexFlatL2(d)  # Use L2 distance
    index.add(scaled_features.astype('float32'))  # Add normalized features to the index
    
    return index, scaled_features

index, scaled_features = build_faiss_index(music)

# Fetch album cover from Spotify
def get_song_album_cover_url(song_name, artist_name):
    try:
        search_query = f"track:{song_name} artist:{artist_name}"
        results = sp.search(q=search_query, type="track")
        if results and results["tracks"]["items"]:
            return results["tracks"]["items"][0]["album"]["images"][0]["url"]
        else:
            return "https://i.postimg.cc/0QNxYz4V/social.png"  # Fallback image
    except Exception as e:
        print(f"Error fetching album cover: {e}")
        return "https://i.postimg.cc/0QNxYz4V/social.png"

# Recommendation Logic
def recommend(song, k=3):
    try:
        # Find the index of the input song
        song_indices = music[music['track_name'] == song].index
        if song_indices.empty:
            print(f"Song '{song}' not found in the dataset.")
            return [], []
        
        index_song = song_indices[0]
        query_vector = scaled_features[index_song].reshape(1, -1).astype('float32')
        
        # Search for k+1 nearest neighbors (including the song itself)
        distances, indices = index.search(query_vector, k + 1)
        
        recommended_music_names = []
        recommended_music_posters = []
        for i in indices[0]:
            if i != index_song:  # Skip the input song itself
                track_name = music.iloc[i]['track_name']
                artist = music.iloc[i]['artists']
                if track_name not in recommended_music_names:  # Avoid duplicates
                    recommended_music_posters.append(get_song_album_cover_url(track_name, artist))
                    recommended_music_names.append(track_name)
                if len(recommended_music_names) == k:  # Stop after k recommendations
                    break
        
        return recommended_music_names, recommended_music_posters
    except Exception as e:
        print(f"Error generating recommendations: {e}")
        return [], []

# Example Usage
print("Dataset loaded successfully!")
selected_song = input("\nEnter a song name to get recommendations: ").strip()

if selected_song in music['track_name'].values:
    recommended_music_names, recommended_music_posters = recommend(selected_song)
    if recommended_music_names:
        print(f"\nIf you like '{selected_song}', you might also enjoy:\n")
        for name, poster in zip(recommended_music_names, recommended_music_posters):
            print(f"- {name}")
            display(Image(url=poster, width=200))
    else:
        print("No recommendations available for the selected song.")
else:
    print("The selected song is not in the dataset.")


Dataset loaded successfully!



Enter a song name to get recommendations:  Say Something


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache



If you like 'Say Something', you might also enjoy:

- Say Something


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import faiss
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import display, Image

# Spotify API Credentials
CLIENT_ID = "4da10aae52c04c8ab5c4a5300212c9d5"
CLIENT_SECRET = "cfc4944866a943c9984b87cce8167b91"

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Load dataset
DATASET_PATH = r"C:\Users\Asmi\Downloads\archive\dataset.csv"
music = pd.read_csv(DATASET_PATH)

# Ensure required columns exist
required_columns = ['track_name', 'artists', 'danceability', 'energy', 'valence', 
                    'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']

for col in required_columns:
    if col not in music.columns:
        raise ValueError(f"Missing column in dataset: {col}")

# Handle missing values
music = music.dropna(subset=required_columns)

# Preprocessing for Similarity Calculation
def build_faiss_index(df):
    features = required_columns[2:]  # Exclude 'track_name' and 'artists'
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    # Build a FAISS index
    d = scaled_features.shape[1]  # Dimensionality of vectors
    index = faiss.IndexFlatL2(d)  # Use L2 distance
    index.add(scaled_features.astype('float32'))  # Add normalized features to the index
    
    return index, scaled_features

index, scaled_features = build_faiss_index(music)

# Fetch album cover from Spotify
def get_song_album_cover_url(song_name, artist_name):
    try:
        search_query = f"track:{song_name} artist:{artist_name}"
        results = sp.search(q=search_query, type="track")
        if results and results["tracks"]["items"]:
            return results["tracks"]["items"][0]["album"]["images"][0]["url"]
        else:
            return "https://i.postimg.cc/0QNxYz4V/social.png"  # Fallback image
    except Exception as e:
        print(f"Error fetching album cover: {e}")
        return "https://i.postimg.cc/0QNxYz4V/social.png"

# Recommendation Logic
def recommend(song, k=3):
    try:
        # Find the index of the input song
        song_indices = music[music['track_name'] == song].index
        if song_indices.empty:
            print(f"Song '{song}' not found in the dataset.")
            return [], []
        
        index_song = song_indices[0]
        query_vector = scaled_features[index_song].reshape(1, -1).astype('float32')
        
        # Search for k+1 nearest neighbors (including the song itself)
        distances, indices = index.search(query_vector, k + 1)
        
        print(f"Debug: Found indices {indices} with distances {distances}")
        
        recommended_music_names = []
        recommended_music_posters = []
        for i in indices[0]:
            if i != index_song:  # Skip the input song itself
                track_name = music.iloc[i]['track_name']
                artist = music.iloc[i]['artists']
                if track_name not in recommended_music_names:  # Avoid duplicates
                    recommended_music_posters.append(get_song_album_cover_url(track_name, artist))
                    recommended_music_names.append(track_name)
                if len(recommended_music_names) == k:  # Stop after k recommendations
                    break
        
        return recommended_music_names, recommended_music_posters
    except Exception as e:
        print(f"Error generating recommendations: {e}")
        return [], []

# Example Usage
print("Dataset loaded successfully!")
selected_song = input("\nEnter a song name to get recommendations: ").strip()

if selected_song in music['track_name'].values:
    recommended_music_names, recommended_music_posters = recommend(selected_song)
    if recommended_music_names:
        print(f"\nIf you like '{selected_song}', you might also enjoy:\n")
        for name, poster in zip(recommended_music_names, recommended_music_posters):
            print(f"- {name}")
            display(Image(url=poster, width=200))
    else:
        print("No recommendations available for the selected song.")
else:
    print("The selected song is not in the dataset.")


Dataset loaded successfully!



Enter a song name to get recommendations:  Give Me Your Forever


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache


Debug: Found indices [[  10  462  199 8391]] with distances [[0.         0.         0.16198781 0.16198781]]


Couldn't read cache at: .cache
Couldn't write token to cache at: .cache



If you like 'Give Me Your Forever', you might also enjoy:

- Give Me Your Forever


- Moonlight, Mistletoe & You


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import faiss
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import display, Image

# Spotify API Credentials
CLIENT_ID = "4da10aae52c04c8ab5c4a5300212c9d5"
CLIENT_SECRET = "cfc4944866a943c9984b87cce8167b91"

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Load dataset
DATASET_PATH = r"C:\Users\Asmi\Downloads\archive\dataset.csv"
music = pd.read_csv(DATASET_PATH)

# Ensure required columns exist
required_columns = ['track_name', 'artists', 'danceability', 'energy', 'valence', 
                    'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'liveness']

for col in required_columns:
    if col not in music.columns:
        raise ValueError(f"Missing column in dataset: {col}")

# Handle missing values
music = music.dropna(subset=required_columns)

# Preprocessing for Similarity Calculation
def build_faiss_index(df):
    features = required_columns[2:]  # Exclude 'track_name' and 'artists'
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    # Build a FAISS index
    d = scaled_features.shape[1]  # Dimensionality of vectors
    index = faiss.IndexFlatL2(d)  # Use L2 distance
    index.add(scaled_features.astype('float32'))  # Add normalized features to the index
    
    return index, scaled_features

index, scaled_features = build_faiss_index(music)

# Fetch album cover from Spotify
def get_song_album_cover_url(song_name, artist_name):
    try:
        search_query = f"track:{song_name} artist:{artist_name}"
        results = sp.search(q=search_query, type="track")
        if results and results["tracks"]["items"]:
            return results["tracks"]["items"][0]["album"]["images"][0]["url"]
        else:
            return "https://i.postimg.cc/0QNxYz4V/social.png"  # Fallback image
    except Exception as e:
        print(f"Error fetching album cover: {e}")
        return "https://i.postimg.cc/0QNxYz4V/social.png"

# Recommendation Logic
def recommend(song, k=3):
    try:
        # Find the index of the input song
        song_indices = music[music['track_name'] == song].index
        if song_indices.empty:
            print(f"Song '{song}' not found in the dataset.")
            return [], []
        
        index_song = song_indices[0]
        query_vector = scaled_features[index_song].reshape(1, -1).astype('float32')
        
        # Search for k+5 nearest neighbors to allow skipping duplicates
        distances, indices = index.search(query_vector, k + 5)
        
        print(f"Debug: Found indices {indices} with distances {distances}")
        
        recommended_music_names = []
        recommended_music_posters = []
        for i in indices[0]:
            if i != index_song:  # Skip the input song itself
                track_name = music.iloc[i]['track_name']
                artist = music.iloc[i]['artists']
                if track_name not in recommended_music_names:  # Avoid duplicates
                    recommended_music_posters.append(get_song_album_cover_url(track_name, artist))
                    recommended_music_names.append(track_name)
                if len(recommended_music_names) == k:  # Stop after k unique recommendations
                    break
        
        return recommended_music_names, recommended_music_posters
    except Exception as e:
        print(f"Error generating recommendations: {e}")
        return [], []


# Example Usage
print("Dataset loaded successfully!")
selected_song = input("\nEnter a song name to get recommendations: ").strip()

if selected_song in music['track_name'].values:
    recommended_music_names, recommended_music_posters = recommend(selected_song)
    if recommended_music_names:
        print(f"\nIf you like '{selected_song}', you might also enjoy:\n")
        for name, poster in zip(recommended_music_names, recommended_music_posters):
            print(f"- {name}")
            display(Image(url=poster, width=200))
    else:
        print("No recommendations available for the selected song.")
else:
    print("The selected song is not in the dataset.")
